In [57]:
import asyncio
import json
from anchorpy import Program, Provider, Wallet, Idl, Context
from solana.rpc.async_api import AsyncClient
from solders.pubkey import Pubkey
from solders.keypair import Keypair
from solders.account import Account

In [ ]:

async def main():
    # Connect to a local Solana cluster (or devnet)
    client = AsyncClient("http://localhost:8899")
    
    # Load your wallet (defaults to ~/.config/solana/id.json)
    wallet = Wallet.local()  # Ensure your wallet is set up appropriately
    provider = Provider(client, wallet)
    
    # Read the IDL file as a raw JSON string
    with open("../../git-solana/target/idl/git_solana.json", "r") as f:
        idl_json_str = f.read()
    
    # Convert the JSON string into an IDL object

    idl = Idl.from_json(idl_json_str)
    print("IDL loaded:", idl)
    
    # Define your program ID using solders Pubkey
    program_id = Pubkey.from_string("5TQo5Bf6yXp9uywEFbp9YKUyveD2pe2LVXRjY2aWRup5")
    
    # Create a program client using the loaded IDL and program ID
    program = Program(idl, program_id, provider)
    
    # Generate a new keypair for the repository account using solders Keypair
    repo_account = Keypair()
    
    # --- Call create_repo ---
    print("Calling create_repo...")

    tx_create = await program.rpc["create_repo"]("My Awesome Repo", ctx=Context(
        accounts={
            "repo": repo_account.pubkey(),  # using solders Keypair property
            "signer": wallet.public_key,   # ensure your wallet.public_key is compatible (or convert if necessary)
            "system_program": Pubkey.from_string("11111111111111111111111111111111"),
        },
        signers=[repo_account]  # The new repo account must sign since it is being created
    ))


    print("Repository created, tx:", tx_create)
    
    # --- Call add_collaborator ---
    # Replace with an actual collaborator public key as needed.
    new_collaborator = Pubkey.from_string("4Kn1P7n6JwGHDm8KvtEickWbA4A3aWzaQn1tNPtyANGh")
    print("Calling add_collaborator...")
    tx_add = await program.rpc["add_collaborator"](new_collaborator, ctx=Context(
        accounts={
            "repo": repo_account.pubkey(),
            "owner": wallet.public_key,
        }
    ))
    print("Collaborator added, tx:", tx_add)
    
    await client.close()


await main()

IDL loaded: Idl(Idl { version: "0.1.0", name: "git_solana", docs: None, constants: [], instructions: [IdlInstruction { name: "createRepo", docs: None, accounts: [IdlAccount(IdlAccount { name: "repo", is_mut: true, is_signer: true, is_optional: None, docs: None, pda: None, relations: [] }), IdlAccount(IdlAccount { name: "signer", is_mut: true, is_signer: true, is_optional: None, docs: None, pda: None, relations: [] }), IdlAccount(IdlAccount { name: "systemProgram", is_mut: false, is_signer: false, is_optional: None, docs: None, pda: None, relations: [] })], args: [IdlField { name: "name", docs: None, ty: String }], returns: None }, IdlInstruction { name: "addCollaborator", docs: None, accounts: [IdlAccount(IdlAccount { name: "repo", is_mut: true, is_signer: false, is_optional: None, docs: None, pda: None, relations: [] }), IdlAccount(IdlAccount { name: "owner", is_mut: true, is_signer: true, is_optional: None, docs: None, pda: None, relations: [] })], args: [IdlField { name: "newCollabo

ValueError: String is the wrong size